# Conexión a base de datos PostgreSQL

Esta libreta establece un ejemplo de conexión a una base de datos PostgreSQL utilizando variables de ambiente

La correcta ejecución de esta libreta incluye los siguientes elementos:
* existe un archivo `.env` con las variables de ambiente de la conexión en la misma ubicación que la libreta
* se dispone de una instancia de servicio PostgreSQL con la base de datos `universidad`

## Conexión a la base de datos

Inicialmente se requiere establecer la conexión con la base de datos. 

Primero se realizan los imports de los módulos requeridos para la tarea

In [2]:
import sys, os

# Importa el conector de la base de datos PostgreSQL
from psycopg2 import connect

# Importa la gestión de errores del conector
from psycopg2 import OperationalError

# Importa la gestión de variables de ambiente
from dotenv import load_dotenv

Se cargan las **variables de ambiente** necesarias para la conexión

In [3]:
# Carga las variables de ambiente del archivo .env
load_dotenv()

PGHOST = os.getenv('PGHOST')
PGPORT = os.getenv('PGPORT')
PGDATABASE= os.getenv('PGDATABASE')
PGUSER = os.getenv('PGUSER') 
PGPASSWORD = os.getenv('PGPASSWORD')

Se inicializa la conexión dentro de un bloque `try..except` para la gestión de **excepciones**

In [4]:
# Establece la conexión con la base de datos
try:
    conn = connect(
                    host = PGHOST,
                    user = PGUSER,
                    dbname = PGDATABASE,
                    password = PGPASSWORD,
                    port = PGPORT)
    
    print('Conectado!')
except OperationalError as err:
    print('Error en la conexión: '+ err)
    
    conn = None

Conectado!


## Cursores

Con la conexión establecida, se crea un **cursor** para realizar una consulta a la base de datos

In [5]:
# Crea el cursor de la conexión y establece la consulta
cur = conn.cursor()

cur.execute("SELECT id_estudiante, nombre, numero_telefono \
            FROM estudiante;")

# Obtiene las filas resultantes de la consulta
cur.fetchmany(size=10)

[(29441979, 'Eduardo', '3764206786'), (1234, 'Evelin', '987456')]

In [6]:
# Obtiene la cantidad de filas afectadas
cur.rowcount

2

A continuación se realiza una **inserción** en la base de datos. 

In [7]:
# Crea el cursor
curInsert = conn.cursor() 

# Establece los valores de los datos a insertar
idEstudiante = "123456"
nombreEstudiante = "María"
numeroTelefono = "+549 9876 123456"

# Ejecuta la acción de inserción con los datos como parámetros de la consulta
curInsert.execute("INSERT INTO estudiante (id_estudiante, nombre, numero_telefono) VALUES (%s, %s, %s)", (idEstudiante, nombreEstudiante, numeroTelefono))

In [8]:
# Obtiene la cantidad de filas afectadas por la operación INSERT
curInsert.rowcount

1

La inserción puede ser confirmada haciendo una nueva consulta a la base de datos

In [9]:
# Crea el cursor de la conexión y establece la consulta
cur = conn.cursor()

cur.execute("SELECT id_estudiante, nombre, numero_telefono \
            FROM estudiante;")

# Obtiene las filas resultantes de la consulta
cur.fetchmany(size=10)

[(29441979, 'Eduardo', '3764206786'),
 (1234, 'Evelin', '987456'),
 (123456, 'María', '+549 9876 123456')]

## Confirmación de cambios y liberación de recursos

Para que los cambios sean persistidos, es necesario confirmar las operaciones

In [10]:
# Confirma las modificaciones a la base de datos
conn.rollback()

Alternativamente al `rollback()`, las modificaciones podrían ser confirmadas mediante la llamada a `commit()`

Finalmente, los recursos de los cursores y la conexión es liberada mediante la llamada a `close()`

In [11]:
# Libera los cursores
cur.close()
curInsert.close()

# Cierra la conexión a la base de datos
conn.close()

## Conclusiones

Mediante el los ejemplos presentados en la libreta se pudo observar en forma práctica cómo realizar una conexión a una base de datos utilizando variables de ambiente. 

Asimismo, se hay realizado consultas aa los datos y se realizó una operación de inserción de datos donde se evidenció la necesidad de confirmar los cambios en la base de datos antes del cierre de la conexión y consecuente liberación de los recursos.